# Detection of Wheezels and Crackles with CNN

### Inspiration: https://www.kaggle.com/eatmygoose/cnn-detection-of-wheezes-and-crackles

En este trabajo nos proponemos clasificar audios respiratorios con anomalias de audios normales. La finalidad del proyecto es poder dar un primer indicio con rapidez de si el paciente presenta un trastorno respiratorio. 

Para entrenar nuestro modelo de deteccion nos basamos en el dataset abierto: https://www.kaggle.com/vbookshelf/respiratory-sound-database

Procederemos en 2 pasos:
- Análisis del dataset presentado
- Entrenamiento del modelo

Imports y librerias


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from IPython.core.display import HTML

HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")


Manejo de archivos

In [ ]:
database_root = "/kaggle/input/respiratory-sound-database"
print("Current path: ", database_root)
print("Main folder contains:")
print(os.listdir(database_root))
auido_files_path = os.path.join(database_root, "Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files")
print(f'Searching from {auido_files_path}')
filenames = [s.split('.')[0] for s in os.listdir(path=auido_files_path) if '.txt' in s]
filenames.sort()
print(f"Found: {len(filenames)} recordings")
# print(filenames)


Informacion general y demográfica de los pacientes tomados para el dataset

In [ ]:
patient_info = pd.read_csv(os.path.join(database_root, "demographic_info.txt"), names=['Patient', 'Age', 'Sex', 'BMI', 'Weight', 'Height'], delim_whitespace=True)
diagnostics = pd.read_csv(os.path.join(database_root, "Respiratory_Sound_Database/Respiratory_Sound_Database/patient_diagnosis.csv"), names=['Patient', 'Diagnosis'])

In [ ]:
patient_info.info()
 = Pneumonia
patient_info.loc[patient_info['Diagnosis'] == ]

## Enfermedades

In [ ]:
patient_info["Diagnosis"] = diagnostics[["Diagnosis"]]
disease_df = patient_info.groupby(['Diagnosis']).count()['Patient'] # Series
disease_df.plot.pie(figsize=(9, 9), explode=(0,0,0,0,0.1,0,0,0), startangle=100, autopct='%1.1f%%')
plt.axis('equal')
plt.title("Diseases", fontsize=22, pad=50.0)
plt.legend(fontsize=12, loc='best', bbox_to_anchor=(1, 1));
plt.show()
print("Total de datos en dataset: ", patient_info['Diagnosis'].count())
print(f'Total de saludables: {patient_info.Diagnosis.value_counts().Healthy}')
print(f'Total de enfermos: {patient_info.Diagnosis.count() - patient_info.Diagnosis.value_counts().Healthy}')

## Age and Health

In [ ]:
Healthy_age = []
Sick_age = []
for index, row in patient_info[['Age','Diagnosis']].iterrows():
    person_age = row['Age']
    if(row['Diagnosis'] != 'Healthy'):
        Sick_age.append(person_age)
    else:
        Healthy_age.append(person_age)

plt.figure(figsize=(11, 9))
colors = ["#90EE90", "#9B870C"]
n, b, p = plt.hist([Healthy_age, Sick_age], bins=15, stacked=True, label=['Healthy', 'Sick'], align="mid", color=colors)
plt.title("Age - Health", fontsize=22, pad=25.0)
plt.xlabel("Age")
plt.ylabel("Count")
plt.xticks(b)
plt.legend(loc="upper right", fontsize=12);

## Género

In [ ]:
sex_df = patient_info.groupby(['Sex']).count()['Patient'] # Series
colors = ["#42A5F5", "#5C6BC0"]
sex_df.plot.pie(figsize=(7, 7), colors=colors, explode=(0.08, 0), shadow=True, startangle=100, autopct='%1.1f%%')
plt.axis('equal')
plt.title("Sex destribution", fontsize=22, pad=25.0)
plt.legend(loc="upper right", fontsize=12);
plt.show()

print("Total de datos en dataset: ", patient_info['Patient'].count())
print(f'Total de hombres: {patient_info.Sex.value_counts().M}')
print(f'Total de mujeres: {patient_info.Sex.value_counts().F}')

## Distribución de Edades y Generos

In [ ]:
F_age = []
M_age = []
for index, row in patient_info[['Age','Sex']].iterrows():
    person_age = row['Age']
    if(row['Sex'] == 'F'):
        F_age.append(person_age)
    else:
        M_age.append(person_age)

colors = ["#42A5F5", "#5C6BC0"]
plt.figure(figsize=(11, 9))
n, b, p = plt.hist([F_age, M_age],histtype="barstacked", bins=15, stacked=True, label=['Female', 'Male'], align="mid", color=colors)
plt.title("Sex - Age Distribution", fontsize=22, pad=25.0)
plt.xlabel("Age")
plt.ylabel("Count")
plt.xticks(b)
plt.legend(loc="upper right", fontsize = 12)

print("Porcentage para cada bin para los hombres:\n", n[1]*round(100/patient_info.Sex.value_counts().M, 1))
print("Porcentage para cada bin para las mujeres:\n", n[0]*round(100/patient_info.Sex.value_counts().F, 1))

Pareceria estar balanceado porcentualmente no? COmo que tenemos graficos muy parecidos para las mujeres que para los hombres, simplemente hay mas hombres que mujeres pero el grafico pareceria ser igual. Esto y el grafico de arriba me llevaria a pensar que no puedo extraer conclusiones de correlacion entre hombres y enfermedad o muejeres y enfermedad ya que tengo las mismas proporciones para 

Graficos que estarian copados hacer: Correlacion entre generos y enfermedad, correlacion entre BMI y enfermedad, Tipos de enfermedad por edad, que enfermedades presentan crackles y whisles (URTI no presenta al parecer), relacion enfermedad - silbidos y crujidos en los audios, cantidad de audios con anomalias

* Tenemos varios caminos:
- 1 Es Distinguir si un audio tiene crujidos o si tiene silbidos
- 2 Es distinguir si un audio tiene cualquiera de los dos o no tienen ninguuno
- 3 Distinguir si el audio es de una persona enferma (una de todas las enfermedades de arriba) o es de una persona sana, mas alla de si tiene o no silbido y crujidos (hay enfermedades que no los presentan)

In [ ]:
patient_info

In [ ]:
def get_recording_info_and_annotations(file_name, root):
    """ Recovers data frames for recording information and annotations """
    fields = file_name.split('_')
    recording_info = pd.DataFrame(data=[fields], columns=['Patient', 'Recording index', 'Chest location', 'Acquisition mode', 'Recording equipment'])
    recording_annotations = pd.read_csv(os.path.join(root, file_name + '.txt'), names = ['Start', 'End', 'Crackles', 'Wheezes'], delimiter= '\t')
    return (recording_info, recording_annotations)

In [ ]:
recording_info_list = []
recording_annotations_list = []
for s in filenames:
    info, annotation = get_recording_info_and_annotations(s, auido_files_path)
    recording_info_list.append(info)
    recording_annotations_list.append(annotation)
    

In [ ]:
recording_annotations_list[0]